# Timing test for IO operation in ChiantiPy
### Author: Mark Cheung, cheung@lmsal.com
## Problem: ChiantiPy.tools.io.wgfaRead() is the main IO bottleneck. 
## Summary: The time it takes to do initialize an ion (e.g. fe14) is dominated by the time spent in wgfaRead().
##          Saving the wgfa files in the HDF5 format (https://www.hdfgroup.org) results in > 300x speed up in wfga read time, and ~ 20x speedup in ion setup time.

In [1]:
import os
import numpy as np
import glob
import h5py
os.environ['XUVTOP']

'/Users/cheung/python/CHIANTI9.0.1'

In [2]:
import ChiantiPy
import ChiantiPy.core as ch
import ChiantiPy.tools.io as io
import ChiantiPy.tools.util as util

 ChiantiPy version 0.9.5 
 found PyQt5 widgets
 using PyQt5 widgets


/Users/cheung/anaconda3/lib/python3.7/site-packages/ChiantiPy-0.9.5-py3.7.egg/ChiantiPy/core/IpyMspectrum.py:9: UserWarning: ipyparallel not found. You won't be able to use the ipymspectrum module
  warnings.warn("ipyparallel not found. You won't be able to use the ipymspectrum module")


## Do some timing tests with IO routines

In [3]:
temp = 10.**(5.8 + 0.05*np.arange(21.))
edens = 1.e+9

In [4]:
%%timeit
fe14 = ch.ion('fe_14', temperature=temp, eDensity=edens, hdf5=False)

1.97 s ± 47.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Why does it take so long? Let's do some IO timing tests.

In [5]:
%%timeit
io.elvlcRead('fe_14')
#This is not a bottleneck

38.9 ms ± 1.06 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [6]:
%%timeit
io.scupsRead('fe_14')
#This is not a bottleneck

14.2 ms ± 149 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [7]:
%%timeit
io.splupsRead('fe_14', filetype='psplups')
#This is not a bottleneck

387 µs ± 4.44 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [8]:
%%timeit
io.wgfaRead('fe_14')
#This is THE bottleneck

2 s ± 35.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
a = io.wgfaRead('fe_14')

### Let's save the output from wgfaRead() into a HDF5 file

In [ ]:
def safe_str(obj):
    return obj.encode('utf-8','ignore')

def wgfa2h5(a,filename):
    h = h5py.File(filename,mode='w')
    for k, v in a.items():
        #print(k, len(v))
        if k == 'ref':
            vv = np.zeros(len(v),dtype=np.string_) #'|S230')
            #There are some strange characters in the comments of the wfga files. Need to encode. 
            for p in range(len(v)):
                vv[p] = safe_str(v[p])
            h.create_dataset(k, data=np.array(vv, dtype=np.string_))
        elif k == 'ionS':
            h.create_dataset(k, data=np.array(v, dtype=np.string_)) #'|S5'))
        elif k == 'filename':
            h.create_dataset(k, data=np.array(v, dtype=np.string_)) #'|S58'))
        elif k == 'pretty1':
            h.create_dataset(k, data=np.array(v, dtype=np.string_)) #'|S18'))
        elif k == 'pretty2':
            h.create_dataset(k, data=np.array(v, dtype=np.string_)) #'|S18'))
        else:
            h.create_dataset(k, data=np.array(v))
    h.close()
    return 

!rm test.h5
print(a.items())
wgfa2h5(a,'test.h5')

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



## Let's try reading in the HDF5 file

In [ ]:
%%timeit
h = h5py.File('test.h5', mode='r')
b = {}
for k in list(h.keys()):
    b[k] = h[k].value
h.close()

3.88 ms ± 81.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# NOTE: wgfaRead() time is 2 to 3 sec, hdf5 read time is a few msec

In [ ]:
base = os.environ['XUVTOP']
atoms = np.sort(glob.glob('{}/*'.format(os.environ['XUVTOP'])))

### The following block is code to convert all wgfa files to HDF5.

In [ ]:
# Convert all wgfa files into HDF5
for atom in atoms:
    astr = (atom.split('/'))[-1]
    ions = (glob.glob("{}/{}/{}*".format(base,astr,astr)))
    #print(astr)
    for ion in ions:
        istr = (ion.split('/'))[-1]
        #print(istr)
        wfiles = glob.glob("{}/{}/{}/*.wgfa".format(base,astr,istr))
        for w in wfiles:
            wgfa = io.wgfaRead(istr, filename=w, total=True)
            filename="{}/{}/{}/{}.h5".format(base,(istr.split('_'))[0], istr, istr)
            wgfa2h5(wgfa, filename)
            print('Wrote out',filename)


Wrote out /Users/cheung/python/CHIANTI9.0.1/al/al_13/al_13.h5
Wrote out /Users/cheung/python/CHIANTI9.0.1/al/al_12/al_12.h5
Wrote out /Users/cheung/python/CHIANTI9.0.1/al/al_9/al_9.h5
Wrote out /Users/cheung/python/CHIANTI9.0.1/al/al_7/al_7.h5
Wrote out /Users/cheung/python/CHIANTI9.0.1/al/al_6/al_6.h5
Wrote out /Users/cheung/python/CHIANTI9.0.1/al/al_8/al_8.h5
Wrote out /Users/cheung/python/CHIANTI9.0.1/al/al_10/al_10.h5
Wrote out /Users/cheung/python/CHIANTI9.0.1/al/al_12d/al_12d.h5
Wrote out /Users/cheung/python/CHIANTI9.0.1/al/al_11/al_11.h5
Wrote out /Users/cheung/python/CHIANTI9.0.1/al/al_3/al_3.h5
Wrote out /Users/cheung/python/CHIANTI9.0.1/al/al_4/al_4.h5
Wrote out /Users/cheung/python/CHIANTI9.0.1/al/al_5/al_5.h5
Wrote out /Users/cheung/python/CHIANTI9.0.1/al/al_2/al_2.h5
Wrote out /Users/cheung/python/CHIANTI9.0.1/ar/ar_10/ar_10.h5
Wrote out /Users/cheung/python/CHIANTI9.0.1/ar/ar_17/ar_17.h5
Wrote out /Users/cheung/python/CHIANTI9.0.1/ar/ar_5/ar_5.h5
Wrote out /Users/cheung/

## Now let's check how long it takes to load an ion using the HDF5 file

In [ ]:
%%timeit
fe14 = ch.ion('fe_14', temperature=temp, eDensity=edens, hdf5=True)

## This is about 20x faster than the original method using WGFA files.